In [1]:
import datetime
from pprint import pprint

import japanize_matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import yfinance as yf
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from stockstats import StockDataFrame

sns.set(font="IPAexGothic", rc={"figure.figsize": (11, 8)})
pd.options.display.float_format = "{:6.2f}".format

/usr/local/lib/python3.9/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
# Valid start and end: YYYY-MM-DD
# Valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
# Valid intervals: [1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo]
response = yf.download(
    tickers="NDAQ",
    period="5y",
    interval="1d",
    group_by="ticker",
)

[*********************100%***********************]  1 of 1 completed


In [3]:
yfdata = response.copy().dropna()
# yfdata = yfdata["1950-01":"202１-12"]  # 直近の暴落を除いて検証する
yfdata

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-05-30,67.48,67.71,67.20,67.42,61.90,618900
2017-05-31,67.68,68.06,67.29,67.65,62.11,1148300
2017-06-01,68.05,68.13,67.38,67.87,62.31,699400
2017-06-02,68.02,68.02,67.37,67.50,61.97,521800
2017-06-05,67.51,68.48,67.51,67.87,62.31,605100
...,...,...,...,...,...,...
2022-05-23,147.45,148.44,145.72,147.71,147.71,828200
2022-05-24,146.43,147.26,144.10,146.63,146.63,779600
2022-05-25,146.22,148.39,145.70,147.50,147.50,879900


In [4]:
def convert_df_to_stock_df(df: pd.DataFrame) -> StockDataFrame:
    sdf = df.copy()
    sdf.rename(
        columns={
            "Open": "open",
            "High": "high",
            "Low": "low",
            "Close": "close",
            "Adj Close": "amount",
            "Volume": "volume",
        },
        inplace=True,
    )
    sdf.index.names = ["date"]
    return StockDataFrame(sdf)


def MACD(arr: pd.DataFrame) -> tuple[pd.Series, pd.Series]:
    sdf = convert_df_to_stock_df(arr)
    StockDataFrame.MACD_EMA_SHORT = 12
    StockDataFrame.MACD_EMA_LONG = 26
    StockDataFrame.MACD_EMA_SIGNAL = 9
    return (sdf["macd"], sdf["macds"])


def RSI(arr: pd.DataFrame, rsi: int) -> pd.Series:
    sdf = convert_df_to_stock_df(arr)
    return sdf["rsi_" + str(rsi)]


# 週足に変換する
def day_2_week(pd: pd.DataFrame) -> pd.DataFrame:
    return (
        pd.copy()
        .resample("W")
        .agg(
            {
                "Open": "first",
                "High": "max",
                "Low": "min",
                "Close": "last",
                "Volume": "sum",
            }
        )
    )

In [24]:
def macd_week(data: pd.DataFrame) -> tuple[pd.Series, pd.Series]:
    m, s = MACD(day_2_week(yfdata))

    # 週間隔のデータなので、日ごとにする
    df = pd.concat([data.copy(), pd.DataFrame([m, s]).transpose()], axis=1)

    tmp_macd = 0
    tmp_macds = 0

    # 一週間に１日だけしかデータが入っていないので、直近のデータをコピーする
    for index, row in df.iterrows():
        if pd.isna(row["macd"]) or pd.isna(row["macds"]):
            df.at[index, "macd"] = tmp_macd
            df.at[index, "macds"] = tmp_macds
        else:
            tmp_macd = row["macd"]
            tmp_macds = row["macds"]

    df = df.dropna()
    return (df["macd"], df["macds"])


def rsi_week(data: pd.DataFrame, during: int) -> pd.Series:
    rsi = RSI(day_2_week(yfdata), during)
    df = pd.concat([data.copy(), pd.DataFrame([rsi]).transpose()], axis=1)

    tmp_rsi = 0
    key = "rsi_" + str(during)
    for index, row in df.iterrows():
        if pd.isna(row[key]):
            df.at[index, key] = tmp_rsi
        else:
            tmp_rsi = row[key]

    df = df.dropna()
    return df[key]

In [44]:
class My_Strategy(Strategy):
    # RSI
    prop_rsi = 7  # 14
    prop_rsi_high = 60  # 70
    prop_rsi_low = 30  # 30

    def init(self):
        self.macd, self.macd_signal = self.I(MACD, self.data.df)
        self.week_macd, self.week_signal = self.I(macd_week, self.data.df)
        self.rsi = self.I(RSI, self.data.df, self.prop_rsi)
        self.rsi_week = self.I(rsi_week, self.data.df, self.prop_rsi)

    def not_trade_with_rsi_range(self):
        return self.prop_rsi_low <= self.rsi[-1] and self.rsi[-1] <= self.prop_rsi_high

    def golden_cross_with_macd_day(self):
        return crossover(self.macd, self.macd_signal)

    def dead_cross_with_macd_day(self):
        return crossover(self.macd_signal, self.macd)

    def is_up_trend(self):
        return self.week_macd[-1] > self.week_signal[-1]

    def next(self):
        # 計算できていない場合トレードしない
        if len(self.data.index) < 7 * 26:  # MACD週足の計算に必要な日数
            return

        # 弱いトレンドの際は注文をしない、RSIより判断する
        if self.not_trade_with_rsi_range():
            return

        # 日足MACDがゴールデンクロスしたら、今までの注文を終了して買い注文
        if self.golden_cross_with_macd_day():
            self.position.close()
            self.buy()
            return

        # 日足MACDがデッドクロスしたら、今までの注文を終了して売り注文
        if self.dead_cross_with_macd_day():
            self.position.close()
            self.sell()
            return


bt = Backtest(yfdata, My_Strategy, cash=10000, commission=0.002, exclusive_orders=True)


# 最適化
# optimize = bt.optimize(
#     # prop_rsi=range(7, 100, 1),
#     # prop_rsi_high=range(20, 80, 2),
#     # prop_rsi_low=range(20, 80, 2),
#     constraint=lambda p: p.prop_rsi_low < p.prop_rsi_high,
#     method="grid",  # unuse model-based optimization
#     maximize="Equity Final [$]",
# )
# bt.plot()
# print(optimize)
# print(optimize._strategy)


# 出力
output = bt.run()
print("Return : " + str(output["Return [%]"]) + "%")
print("Trades : " + str(output["# Trades"]))
print(output)
bt.plot(filename='macd_and_rsi')

Return : 129.18720145721443%
Trades : 30
Start                     2017-05-30 00:00:00
End                       2022-05-27 00:00:00
Duration                   1823 days 00:00:00
Exposure Time [%]                       85.16
Equity Final [$]                     22918.72
Equity Peak [$]                      24326.77
Return [%]                             129.19
Buy & Hold Return [%]                  130.85
Return (Ann.) [%]                       18.04
Volatility (Ann.) [%]                   30.16
Sharpe Ratio                             0.60
Sortino Ratio                            1.06
Calmar Ratio                             0.47
Max. Drawdown [%]                      -38.52
Avg. Drawdown [%]                       -4.22
Max. Drawdown Duration      289 days 00:00:00
Avg. Drawdown Duration       30 days 00:00:00
# Trades                                   30
Win Rate [%]                            70.00
Best Trade [%]                          18.37
Worst Trade [%]                        

Row(id='28463', ...)

In [43]:
bt.plot??

Signature:
bt.plot(
    *,
    results: pandas.core.series.Series = None,
    filename=None,
    plot_width=None,
    plot_equity=True,
    plot_return=False,
    plot_pl=True,
    plot_volume=True,
    plot_drawdown=False,
    smooth_equity=False,
    relative_equity=True,
    superimpose: Union[bool, str] = True,
    resample=True,
    reverse_indicators=False,
    show_legend=True,
    open_browser=True,
)
Docstring:
Plot the progression of the last backtest run.

If `results` is provided, it should be a particular result
`pd.Series` such as returned by
`backtesting.backtesting.Backtest.run` or
`backtesting.backtesting.Backtest.optimize`, otherwise the last
run's results are used.

`filename` is the path to save the interactive HTML plot to.
By default, a strategy/parameter-dependent file is created in the
current working directory.

`plot_width` is the width of the plot in pixels. If None (default),
the plot is made to span 100% of browser width. The height is
currently non-adjusta